In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, BatchNormalization, Activation, Add, UpSampling2D, Concatenate, 
from tensorflow.keras.models import Model
from tensorflow.keras import layers

ImportError: ignored

In [ ]:
input_shape = (256, 256, 3)
num_classes = 3

In [ ]:
def DeeplabV3Plus(input_shape, num_classes):
    # Input layer
    img_input = layers.Input(shape=input_shape)
    x = img_input

    # Feature extractor
    backbone = tf.keras.applications.Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    skip0 = backbone.get_layer('block1_conv1_act').output
    skip1 = backbone.get_layer('block3_sepconv2_bn').output
    x = backbone.output

    # ASPP module
    x = layers.AtrousConvolution2D(filters=256, kernel_size=3, dilation_rate=6, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate=0.5)(x)

    x = layers.AtrousConvolution2D(filters=256, kernel_size=3, dilation_rate=12, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate=0.5)(x)

    x = layers.AtrousConvolution2D(filters=256, kernel_size=3, dilation_rate=18, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate=0.5)(x)

    # Upsampling module
    x = layers.UpSampling2D(size=(4, 4))(x)
    x = layers.Convolution2D(filters=48, kernel_size=1, padding='same')(x)

    skip1 = layers.Convolution2D(filters=48, kernel_size=1, padding='same')(skip1)
    x = layers.Concatenate()([x, skip1])
    x = layers.SeparableConv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate=0.5)(x)

    x = layers.UpSampling2D(size=(4, 4))(x)
    x = layers.Convolution2D(filters=32, kernel_size=1, padding='same')(x)

    skip0 = layers.Convolution2D(filters=32, kernel_size=1, padding='same')(skip0)
    x = layers.Concatenate()([x, skip0])
    x = layers.SeparableConv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate=0.5)(x)

    # Output layer
    x = layers.Convolution2D(num_classes, (1, 1), padding='same')(x)
    x = layers.UpSampling2D(size=(4, 4))(x)

    # Final model
    output_layer = layers.Activation('softmax')(x)
    model = Model(img_input, output_layer, name='DeeplabV3Plus')

    return model

In [ ]:
model = DeeplabV3Plus(input_shape, num_classes)

83683744/83683744 [==============================] - 1s 0us/step


AttributeError: ignored